# Functional Programming

In the functional programming paradigm, functions are the key building block. More concretely, the focus is on pure functions, avoiding shared state, mutable data, and side-effects.

To get an example, take the construction of a list. You can do it two ways:

In [ ]:
a = []
for i in 10:15
    if isodd(i)
        push!(a, i*i)
    end
end
a

In [ ]:
a = [i*i for i in 10:15 if isodd(i)]

**Question to you:** Which one do you think is more functional in the above sense?

# Sideeffects

The probably most famous side effect are errors and error handling respectively.

In [ ]:
function checkoddness_errorsideeffect(i)
    if isodd(i)
        return i
    else
        error("Found even number $i")
    end
end

In [ ]:
try
    checkoddness_errorsideeffect(4)
catch exc
    exc
end

**Question to you:** How could we get rid of this sideeffect?

# Functors - the basics of sideffects

One famous answer is to make it explicit and include it into the returned type.

For example, instead a dealing with a return value which may just not exist but fail instead, we deal with a return value that may be an exception.

For more control we wrap this into our own litte helper type.

In [ ]:
struct MyTry
    value
end

In [ ]:
function checkoddness_mytry(i)
    if isodd(i)
        return MyTry(i)
    else
        MyTry(ErrorException("Found even number $i"))
    end
end

In [ ]:
checkoddness_mytry(10)

In [ ]:
checkoddness_mytry(1)

I hope you get the functional feeling: now nothing is left which can surprise you, everything is explicit.

(This is very important for production systems, as they tend to be large, and the larger you get, the less you want to be surprised by some little part doing unexpected things.)

-------------

However we cannot easily work on `MyTry` yet. Let's rebuild the sideeffect.

In [ ]:
function apply_program_within_the_MyTry_wrapper(a_function_representing_my_program, mytry)
    if isa(mytry.value, Exception)
        # shortcycle if we already found an error
        return mytry
    else
        # otherwise apply the program to the value and wrap it again into MyTry
        return MyTry(a_function_representing_my_program(mytry.value))
    end
end

In [ ]:
a = checkoddness_mytry(3)
apply_program_within_the_MyTry_wrapper(i -> i*i, a)

There is indeed a common name for this function, it is called `map`.

In [ ]:
# it is the same definition as above, just slightly differently written
function Base.map(func, mytry::MyTry)
    isa(mytry.value, Exception) && return mytry
    MyTry(func(mytry.value))
end

and using do-notation we already have a quite nice syntax to work within our effect

In [ ]:
a = checkoddness_mytry(11)
map(a) do i
    i*i
end

What have we done?
1. instead of using plain errors, we defined our own return type which makes clear that it may contain an Exception
2. we wrote a higher level function which knows about this further wrapper and can apply a function just as if it would now about the error handling

Congrats 🙂 ! You've just build your first example of what is called a **Functor** in functional programming.

-------

But you may also say, despite the do and the map, it still looks more complicated than before, which it is.

And to admit, it is also less flexible, because you cannot yet combine multiple errors easily. You need to do something like this.

In [ ]:
a = checkoddness_mytry(11)
map(a) do i
    b = checkoddness_mytry(i + 3)
    map(b) do j
        j^3
    end
end

We can massively improve over this.

# Monads - sequential sideffects

So what is our goal again?
> We have multiple computations which return MyTry (Functors in general), let's call them effectful computations, and we want to combine them together.

Like above, where we just wanted to called `checkoddness_mytry` again on the return value of the first call.

-------

It turns out, what we need is just a way to flatten these nested `MyTry(MyTry(...))`. Instead of flattening them afterwards, what is commonly done is to define version of `map` which directly flattens out `MyTry(MyTry(...))` to `MyTry(...)`. It is typically called `flatmap`.

Let's define `flatmap` for `MyTry` and it should get clear.

In [ ]:
function flatmap(func_returning_mytry, mytry::MyTry)
    if isa(mytry.value, Exception)
        # in case of an error, the short cycling is identical to `map`,
        # which indeed returns a plain `MyTry(...)`, and not a `MyTry(MyTry(...)`, exactly like we wished
        return mytry
    else
        # in the case where we have a proper value, we just apply the given function and return its result
        # again this will be a plain `MyTry(...)`, and not a `MyTry(MyTry(...)`, exactly like we wished
        return func_returning_mytry(mytry.value)
    end
end 

That is it 🙂 . Congrats again - you understood your first Monad! 🙂

Let's see how it works

In [ ]:
a = checkoddness_mytry(11)
flatmap(a) do i
    b = checkoddness_mytry(i + 3)
    map(b) do j
        j^3
    end
end

Yeah! we now have a `MyTry` and not a `MyTry(MyTry(...))` any longer!

You won't believe it, but indeed this is everything. Now you are a master of this black magic which captures sideeffects cleanly and makes you able to work on them smoothly.

*Cough, smoothly?*

Yes, the concept is so powerful that we now can add nice syntax upon it by using a little package. (DISCLAIMER: I am the author of that package).

In [ ]:
using Monadic

In [ ]:
@monadic map flatmap begin
    i = checkoddness_mytry(11)
    j = checkoddness_mytry(i + 2)
    @pure i + j*j
end

I truly hope this mindblows you.

# What about Lists?

We have developed everything using Errors as one example of sideeffects. Actually the concept is not restricted at all to sideeffects. (It is only that sideffects have been one of the motivators in programming theory which lead to Functors and Monads finally being developed as such).

**We only need to define `map` and `flatmap` and we are ready to go.**

Let's take the list again. We already know what `map` should do - applying a function elementwise.

In [ ]:
map([1,2,3,4]) do i
    i * i
end

Works out of the box.

So what should flatmap look like? Just concatenating the sublists respectively.

In [ ]:
flatmap(func_returning_vector, vector::Vector) = vcat(map(func_returning_vector, vector)...)

And ready we are!

In [ ]:
@monadic map flatmap begin
    i = [1, 3, 9]
    j = [i, i*i]
    @pure i + j
end

What does this monad do? It behaves like nested for loops, however not only executing things, but collecting all results into a final list.

# Outlook Monads

Congratulations. You understood Monads, something I would guess 90% of all programmers do not understand, probably more. It is a simple and super-mighty concept, and production-ready of course.

There are many more common Monads which are used in production. Just that you heard some names: There is Maybe/Option, Either, Try, Continuation Monad, AtomicComputation Monads, Probabilistic Indeterminism Monads, ContextManagers can be represented as Monads, there are Writer, Reader, and Free Monads, and many more.

As Monads have a unified interface, there are also helpers which work generically on any Monad.

There is also a typical pitfall of monads, namely while you can easily combine a single monad with itself (that is kind of its definition), you cannot easily combine multiple different monads. However, just that you know, there are production ready solution to this, and one of it are so called Algebraic Effects, which are not the same as Monads, but similar enough for most usecases, and indeed can combine different Effects seamlessly. I guess only 0.1% of the programmers know about this one, if at all.

# Applicatives - parallel sideeffects

We have no time left, however you should at least know that this concept exist, it is very wide spread.

The key difference to Monads is that monads always need to know the precise outcome of the previous effectful computation, and hence they need to be executed sequentially. Applicatives don't have this requirement and can indeed be executed in a parallel manner.

Key functions:
- Functor: `map`
- Monad: `flatmap`
- Applicative: `mapn`

With `mapn` I mean a higher order function similar to map, however it can take multiple arguments at once.

```julia
a = func_with_mytry(11)
b = func_with_mytry(12)
mapn(a, b) do i, j
    i + j
end
```

--------

You can implement such `mapn` also using `flatmap`, however then it would be sequential by definition. Defining `mapn` directly instead, enables you to parallelize the effects executions instead.


To make this clear, consider a list as the monad. Let's have two lists where we just want to find all pairs.
```julia
list1 = [1, 2]
list2 = [6, 7, 8]
```
With monads, i.e. `flatmap`, you would implement `mapn` like follows:
```julia
function mapn_monadic(func, list1, list2)
    flatmap(list1) do i
        map(list2) do j
            func(i, j)
        end
    end
end
```
Studying this, you see, that we go through `list1` one by one, everytime go over `list2` again and again. It is sequential in nature, the execution over `list2` really depends on the concrete value of `list1` which we are currently looking at.

With applicatives we can imagine whatever implementation we guess is better, and also inspecting both lists together.
```julia
function mapn(func, list1::Vector, list2::Vector)
    map([(i, j) for i in list1 for j in list2]) do (i, j)
       func(i,j)
    end
end
```
Studying this you see, that we can first construct the pairs, and then parallelize the whole computation over all pairs.
Replacing `map` with `Distributed.pmap`, which is like map, but uses all available worker nodes, you get a true feeling of the parallelism.